In [1]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import os, math

from lifelines.utils import concordance_index
from lifelines import CoxPHFitter

from lifelines.datasets import load_rossi
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import r2_score

In [2]:
np.random.seed(15)
torch.manual_seed(15)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
data=pd.read_csv('./dataset/latent_features_lung1.csv')
data2=data.drop(['PatientID','Survival.time','Deadstatus.event'], axis=1)
E=np.array(data["Deadstatus.event"])
Y=np.array(data["Survival.time"])
X=np.array(data2)
X=X.astype('float32')
E=E.astype('float32')

# y,x =X.shape
# normalized_df = np.zeros((y,x))
# for i in range(x):
#     for j in range(y):
#         X[j,i]=(X[j,i]-X[:,i].min())/(X[:,i].max()-X[:,i].min())



X_train,X_val,Y_train,Y_val=train_test_split(X,Y,test_size=0.25, random_state=0) # no random
X_train,X_val,E_train,E_val=train_test_split(X,E,test_size=0.25, random_state=0) # no random


#Standardize
# scaler=preprocessing.StandardScaler().fit(X_train[:,[1,6]])
# X_train[:,[1,6]]=scaler.transform(X_train[:,[1,6]])
# X_val[:,[1,6]]=scaler.transform(X_val[:,[1,6]])

#Sorting for NNL!
sort_idx = np.argsort(Y_train)[::-1]
X_train=X_train[sort_idx]
Y_train=Y_train[sort_idx]
E_train=E_train[sort_idx]

sort_idx = np.argsort(Y_val)[::-1]
X_val=X_val[sort_idx]
Y_val=Y_val[sort_idx]
E_val=E_val[sort_idx]


y_train_mean = np.mean(Y_train)

X_train = torch.Tensor(X_train).to(device)
Y_train = torch.Tensor(Y_train).to(device)
E_train = torch.Tensor(E_train).to(device)

X_val = torch.Tensor(X_val).to(device)
Y_val = torch.Tensor(Y_val).to(device)
E_val = torch.Tensor(E_val).to(device)

print("X-train shape: ", X_train.shape)
print("Y-train shape: ", Y_train.shape)
print("E-train shape: ", E_train.shape)
print("y train mean: ", y_train_mean)


X-train shape:  torch.Size([288, 512])
Y-train shape:  torch.Size([288])
E-train shape:  torch.Size([288])
y train mean:  76.74503968457985


In [4]:
#Loss Function
    
class PartialLogLikelihood(nn.Module):
    def __init__(self):
        super(PartialLogLikelihood, self).__init__()
    def forward(self, logits, fail_indicator, ties):
        '''
        fail_indicator: 1 if the sample fails, 0 if the sample is censored.
        logits: raw output from model 
        ties: 'noties' or 'efron' or 'breslow'
        '''
        logL = 0
        # pre-calculate cumsum
        #cumsum_y_pred = torch.cumsum(logits, 0)
        hazard_ratio = torch.exp(logits)
        cumsum_hazard_ratio = torch.cumsum(hazard_ratio, 0)
        if ties == 'noties':
            log_risk = torch.log(cumsum_hazard_ratio)
            likelihood = logits - log_risk
            # dimension for E: np.array -> [None, 1]
            uncensored_likelihood = likelihood * fail_indicator
            logL = -torch.sum(uncensored_likelihood)
        else:
            raise NotImplementedError()
        # negative average log-likelihood
        observations = torch.sum(fail_indicator, 0)
        return 1.0*logL / observations  

class NewLoss(nn.Module):
    def __init__(self):
        super(NewLoss, self).__init__()
    def forward(self, logits, cumsum_hazard_ratio):
        '''
        fail_indicator: 1 if the sample fails, 0 if the sample is censored.
        logits: raw output from model 
        ties: 'noties' or 'efron' or 'breslow'
        '''
        logL = 0
        # pre-calculate cumsum
        log_risk = torch.log(cumsum_hazard_ratio)
        likelihood = logits - log_risk

        return -likelihood    
#model
class DeepCPH(nn.Module):
    def __init__(self):
        super(DeepCPH, self).__init__()
        self.linear1 = nn.Linear(512, 64, bias=True)
        self.linear2 = nn.Linear(64,32, bias=True)
        self.linear3 = nn.Linear(32, 8, bias=True)
        self.linear4 = nn.Linear(8, 1, bias=True)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3, inplace=True)
        
        self.reset_parameters()
    
    def reset_parameters(self):
        for weight in self.parameters():
            stdv = 1.0 / math.sqrt(weight.size(0))
            torch.nn.init.uniform_(weight, -stdv, stdv)
            
    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear4(x)
        return x
    
Deep_CPH_model = DeepCPH()
Deep_CPH_model.to(device)
optimizer = torch.optim.Adam(Deep_CPH_model.parameters(), lr=0.001, weight_decay=0.0)
criterion = PartialLogLikelihood()
#criterion = NewLoss()

In [5]:
# # train Deep Cox Proportional Hazard Network
for epoch in range(1000):
    Deep_CPH_model.train()
    optimizer.zero_grad()
    y_pred = Deep_CPH_model(X_train)
    loss = criterion(y_pred, E_train, 'noties')
#     if loss < 900:
#         print(epoch, loss)
#         break
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(epoch, loss.item())
# train Deep Cox Proportional Hazard Network
# for epoch in range(10):
#     Deep_CPH_model.train()
#     sum_h, loss_train = 0, 0
#     n_fail_indicator = 0
    
#     for i in range(len(X_train)):
        
#         y_pred = Deep_CPH_model(X_train[i])
#         sum_h += np.exp(y_pred.cpu().detach().numpy())
        
#         if E_train[i] == 0:
#             continue
#         else:
#             n_fail_indicator +=1
#         loss = criterion(y_pred, torch.Tensor(sum_h).to(device)) #/ n_fail_indicator
#         #loss.backward()
#         loss_train += loss
#     optimizer.zero_grad()
#     loss_train.backward()
#     optimizer.step()
#     if loss_train < 550:
#         print(epoch, loss_train)
#         break
    
#     #if epoch % 10 == 0:
#     print(epoch, loss_train)
#     loss_train=0

0 1400.9349365234375
50 1346.798828125
100 1346.7362060546875
150 1346.2557373046875
200 1346.1527099609375
250 1346.78125
300 1346.5859375
350 1346.8612060546875
400 1346.5550537109375
450 1346.6495361328125
500 1346.5267333984375
550 1346.7286376953125
600 1346.6572265625
650 1347.4324951171875
700 1346.7542724609375
750 1347.1185302734375
800 1346.0799560546875
850 1346.9049072265625
900 1346.5799560546875
950 1346.5506591796875


In [6]:
#predict
with torch.no_grad():
    hazard_rate_pred = Deep_CPH_model(X_train)
    hr_pred = hazard_rate_pred.cpu().detach().numpy()
    hr_pred2 = hr_pred - np.mean(hr_pred)
    general_prediction=np.random.exponential(scale=y_train_mean)/np.exp(hr_pred2)
    print(general_prediction.max())
    general_prediction[general_prediction >= 800] = 800


146.82866


In [7]:
def concordance_correlation_coefficient(y_true, y_pred,
                       sample_weight=None,
                       multioutput='uniform_average'):
    """Concordance correlation coefficient.
    The concordance correlation coefficient is a measure of inter-rater agreement.
    It measures the deviation of the relationship between predicted and true values
    from the 45 degree angle.
    Read more: https://en.wikipedia.org/wiki/Concordance_correlation_coefficient
    Original paper: Lawrence, I., and Kuei Lin. "A concordance correlation coefficient to evaluate reproducibility." Biometrics (1989): 255-268.  
    Parameters
    ----------
    y_true : array-like of shape = (n_samples) or (n_samples, n_outputs)
        Ground truth (correct) target values.
    y_pred : array-like of shape = (n_samples) or (n_samples, n_outputs)
        Estimated target values.
    Returns
    -------
    loss : A float in the range [-1,1]. A value of 1 indicates perfect agreement
    between the true and the predicted values.
    Examples
    --------
    >>> from sklearn.metrics import concordance_correlation_coefficient
    >>> y_true = [3, -0.5, 2, 7]
    >>> y_pred = [2.5, 0.0, 2, 8]
    >>> concordance_correlation_coefficient(y_true, y_pred)
    0.97678916827853024
    """
    cor = np.corrcoef(y_true, y_pred)[0][1]
    
    mean_true=np.mean(y_true)
    mean_pred=np.mean(y_pred)
    
    var_true=np.var(y_true)
    var_pred=np.var(y_pred)
    
    sd_true=np.std(y_true)
    sd_pred=np.std(y_pred)
    
    numerator=2*cor*sd_true*sd_pred
    
    denominator=var_true+var_pred+(mean_true-mean_pred)**2

    return numerator/denominator

c = concordance_correlation_coefficient(Y_train.cpu().detach().numpy(), general_prediction.reshape(-1))
print(Y_train.cpu().detach().numpy().min(), Y_train.cpu().detach().numpy().max(), Y_train.cpu().detach().numpy().mean())
print(general_prediction.reshape(-1).min(), general_prediction.reshape(-1).max(), general_prediction.reshape(-1).mean())
print("c: ", c)

1.4285715 309.2857 76.74504
133.90419 146.82866 144.99902
c:  -0.0004215573677081877


In [8]:
class MSE(nn.Module):
    def __init__(self):
        super(MSE, self).__init__()
    def forward(self, y_true, y_pred):
        loss = (abs(y_true - y_pred)).sum() / y_true.shape[0]
        return loss  

#model
class Distribution_network(nn.Module):
    def __init__(self):
        super(Distribution_network, self).__init__()
        self.linear1 = nn.Linear(512, 128, bias=True)
        self.linear2 = nn.Linear(128,32, bias=True)
        self.linear3 = nn.Linear(32, 16, bias=True)
        self.linear4 = nn.Linear(16, 1, bias=True)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3, inplace=True)
        self.reset_parameters()
    
    def reset_parameters(self):
        for weight in self.parameters():
            stdv = 1.0 / math.sqrt(weight.size(0))
            torch.nn.init.uniform_(weight, -stdv, stdv)
            
    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear3(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear4(x)
        return x
    
survival_model = Distribution_network()
survival_model.to(device)
#optimizer = torch.optim.Adam(survival_model.parameters(), lr=0.00001)
optimizer = torch.optim.RMSprop(survival_model.parameters(), lr=0.1, alpha=0.99, eps=1e-08)
criterion = MSE()

In [9]:
Deep_CPH_model.eval()

X_train = X_train[E_train==1]
Y_train = Y_train[E_train==1]
X_val = X_val[E_val==1]
Y_val = Y_val[E_val==1]

with torch.no_grad():
    hazard_rate_pred = Deep_CPH_model(X_train)
# train Distribution function network
min_mae = 1000
model_min = []
for epoch in range(776):
    survival_model.train()
    optimizer.zero_grad()
    y_pred = survival_model(X_train) / hazard_rate_pred.exp()
    print(y_pred)
    loss = criterion(Y_train, y_pred)
    loss.backward()
    optimizer.step()
#     if epoch % 100 == 0:
#         print(epoch, loss.item())
        #predict
    with torch.no_grad():
        survival_model.eval()
        hazard_rate_pred_val = Deep_CPH_model(X_val)
        predicted_survival = survival_model(X_val) / hazard_rate_pred_val.exp()
        mae = abs(predicted_survival.reshape(-1)-Y_val).mean().item()
        if mae <min_mae:
            min_mae = mae
            print("Min: ", min_mae, epoch)

tensor([[ 0.2376],
        [-0.8747],
        [-0.4230],
        [ 1.5318],
        [ 0.2448],
        [ 0.7265],
        [ 0.6394],
        [-1.0726],
        [ 0.2011],
        [ 0.6987],
        [-0.7759],
        [ 0.5047],
        [-0.7979],
        [ 1.1209],
        [-1.1680],
        [-0.3483],
        [-1.9642],
        [-0.5815],
        [-0.5168],
        [-0.4312],
        [-0.0716],
        [ 0.8899],
        [-0.3236],
        [-0.3769],
        [ 0.1404],
        [-1.4509],
        [-1.8924],
        [-0.8685],
        [ 0.0096],
        [ 1.9803],
        [ 1.3226],
        [ 0.1368],
        [ 2.8115],
        [-0.3846],
        [ 0.5865],
        [-2.1514],
        [ 0.3580],
        [-0.4619],
        [-1.3344],
        [-1.2824],
        [ 1.1980],
        [-1.2024],
        [ 1.2680],
        [-0.4674],
        [-1.3981],
        [ 1.0568],
        [-0.2145],
        [-0.1668],
        [ 0.2986],
        [-0.3608],
        [-0.3343],
        [ 0.8136],
        [-0.

Min:  57.06182098388672 3
tensor([[71.4000],
        [17.4043],
        [ 1.5966],
        [ 5.0274],
        [ 1.5966],
        [45.5495],
        [ 1.5966],
        [ 3.1108],
        [ 2.6755],
        [11.3211],
        [ 4.4492],
        [ 3.4893],
        [ 4.2680],
        [ 3.2027],
        [42.4422],
        [ 6.2141],
        [ 1.5966],
        [ 5.4056],
        [49.5334],
        [ 4.4051],
        [15.9375],
        [ 1.5966],
        [ 5.7307],
        [17.1847],
        [ 1.5966],
        [66.1072],
        [ 1.5966],
        [ 1.5966],
        [ 2.4971],
        [19.9206],
        [ 2.6434],
        [11.6056],
        [ 3.5513],
        [22.7035],
        [ 1.5966],
        [ 6.8814],
        [ 4.3292],
        [17.2006],
        [ 2.6010],
        [ 3.4636],
        [ 1.5966],
        [ 1.5966],
        [ 2.1821],
        [33.1562],
        [ 2.4497],
        [ 4.1092],
        [18.2903],
        [ 1.5966],
        [ 1.5966],
        [ 2.4040],
        [ 4.3863],
     

tensor([[11.7746],
        [10.3754],
        [10.0686],
        [11.8471],
        [ 7.0013],
        [ 7.9093],
        [11.3272],
        [15.9885],
        [10.6265],
        [ 9.7628],
        [ 9.9444],
        [10.8578],
        [14.0559],
        [ 7.8637],
        [ 6.1472],
        [14.1261],
        [10.7588],
        [ 9.0117],
        [ 5.4864],
        [13.1970],
        [ 6.5433],
        [ 9.8857],
        [ 8.3105],
        [ 5.1753],
        [ 3.0346],
        [ 8.7733],
        [10.8983],
        [14.1360],
        [ 8.4452],
        [ 7.9495],
        [ 2.8091],
        [15.0366],
        [ 9.4696],
        [ 9.8971],
        [11.6654],
        [ 9.7461],
        [ 9.7892],
        [13.2701],
        [ 5.8598],
        [ 9.9184],
        [ 9.0731],
        [ 6.1846],
        [ 6.7082],
        [ 5.2980],
        [ 4.9662],
        [13.3473],
        [ 9.6502],
        [ 4.2732],
        [10.9417],
        [11.2115],
        [10.3137],
        [ 7.1371],
        [ 8.

Min:  45.30775451660156 11
tensor([[18.3282],
        [22.3059],
        [17.7643],
        [17.1501],
        [17.2447],
        [21.3220],
        [20.0147],
        [ 9.5228],
        [13.8213],
        [14.0320],
        [11.6014],
        [ 8.7854],
        [17.3149],
        [ 8.9823],
        [ 8.4925],
        [10.9021],
        [11.6753],
        [13.0735],
        [18.0028],
        [18.7330],
        [21.0253],
        [18.8104],
        [13.9642],
        [13.0874],
        [17.2292],
        [20.2204],
        [12.9890],
        [13.1494],
        [21.4381],
        [16.2689],
        [18.2854],
        [14.6193],
        [17.5865],
        [22.7448],
        [18.1964],
        [ 8.2341],
        [16.1574],
        [21.1432],
        [16.3002],
        [18.0631],
        [17.9517],
        [11.5770],
        [ 9.3050],
        [19.9117],
        [14.6011],
        [12.3374],
        [18.5207],
        [17.3313],
        [14.0013],
        [23.3806],
        [13.1345],
    

tensor([[16.9836],
        [22.2156],
        [22.0937],
        [22.6535],
        [25.2390],
        [29.8370],
        [25.9271],
        [23.5138],
        [16.7326],
        [20.3365],
        [29.9679],
        [17.3669],
        [24.9406],
        [18.7188],
        [12.3139],
        [12.4961],
        [16.9043],
        [24.4939],
        [26.0135],
        [25.8149],
        [21.0210],
        [14.7357],
        [25.6682],
        [22.7122],
        [26.1977],
        [23.9066],
        [12.1500],
        [25.5776],
        [20.2345],
        [18.1625],
        [23.9561],
        [30.0804],
        [10.7265],
        [ 8.6046],
        [21.2547],
        [23.5090],
        [18.8428],
        [21.2884],
        [29.5795],
        [11.5959],
        [20.4630],
        [ 4.9150],
        [14.6201],
        [17.1677],
        [23.0505],
        [12.1707],
        [13.1651],
        [19.7530],
        [14.8086],
        [29.7166],
        [13.6142],
        [12.3746],
        [13.

...

tensor([[43.7079],
        [41.8384],
        [41.9479],
        [44.0881],
        [41.7820],
        [42.7033],
        [42.4633],
        [44.1659],
        [43.2399],
        [42.8547],
        [44.4019],
        [43.8641],
        [42.6454],
        [41.5037],
        [43.6720],
        [42.2191],
        [42.0869],
        [43.7653],
        [43.7624],
        [42.7415],
        [41.2197],
        [42.5576],
        [40.8253],
        [42.9632],
        [41.4229],
        [41.8173],
        [41.7750],
        [42.8214],
        [43.7633],
        [40.1279],
        [43.5496],
        [43.4625],
        [43.0115],
        [41.6470],
        [41.7932],
        [40.7187],
        [42.6893],
        [41.3922],
        [42.1833],
        [42.7399],
        [41.9404],
        [42.6025],
        [42.6080],
        [40.8766],
        [41.2083],
        [43.5862],
        [43.9311],
        [42.3460],
        [42.0192],
        [41.6273],
        [44.7794],
        [45.0790],
        [42.

Min:  34.499210357666016 761
tensor([[41.2198],
        [44.0721],
        [43.3207],
        [40.9232],
        [44.2299],
        [40.8587],
        [45.1266],
        [42.7907],
        [41.9692],
        [43.2610],
        [40.5979],
        [39.7738],
        [43.2008],
        [43.3969],
        [43.4136],
        [41.1961],
        [43.0620],
        [41.2547],
        [42.1924],
        [42.1820],
        [43.3707],
        [43.1440],
        [42.6949],
        [42.8280],
        [44.0920],
        [44.9999],
        [43.2218],
        [43.9851],
        [42.2118],
        [41.6763],
        [42.2394],
        [42.7730],
        [40.3088],
        [43.5870],
        [43.7749],
        [45.1985],
        [41.7491],
        [41.8663],
        [42.5476],
        [44.2221],
        [43.1724],
        [41.8586],
        [41.3949],
        [44.9812],
        [40.3286],
        [43.1727],
        [40.7850],
        [43.0174],
        [44.7377],
        [42.9175],
        [40.6941],
  

tensor([[43.0225],
        [41.7837],
        [42.3935],
        [41.6062],
        [41.5389],
        [42.2342],
        [42.3322],
        [41.6690],
        [42.8063],
        [42.2764],
        [42.4222],
        [42.4473],
        [41.3646],
        [43.6539],
        [42.4305],
        [42.9483],
        [43.8651],
        [42.6159],
        [42.1962],
        [39.6643],
        [42.7952],
        [41.5166],
        [40.1597],
        [44.3491],
        [43.0081],
        [41.3444],
        [42.2559],
        [43.1216],
        [42.3489],
        [43.2594],
        [42.2310],
        [42.5685],
        [42.1498],
        [42.3126],
        [40.5279],
        [42.7981],
        [41.9190],
        [41.2202],
        [42.6992],
        [42.2586],
        [41.8892],
        [41.7366],
        [42.8371],
        [42.5288],
        [41.8919],
        [40.1754],
        [43.1649],
        [40.1452],
        [44.5418],
        [42.4565],
        [41.7070],
        [41.5985],
        [42.

tensor([[42.1284],
        [43.2878],
        [41.4288],
        [42.7876],
        [43.3393],
        [43.7126],
        [43.6020],
        [41.0559],
        [41.9876],
        [42.0978],
        [42.0130],
        [42.4093],
        [41.1892],
        [42.7142],
        [43.8001],
        [41.3327],
        [42.7099],
        [41.8753],
        [42.7736],
        [42.5153],
        [43.1162],
        [45.0010],
        [42.2917],
        [42.0791],
        [40.8792],
        [43.0288],
        [41.5480],
        [43.8672],
        [42.9141],
        [42.3769],
        [44.6987],
        [43.7069],
        [42.9961],
        [41.3067],
        [40.5098],
        [42.7849],
        [43.8650],
        [42.5775],
        [40.0409],
        [42.5906],
        [40.9406],
        [41.4855],
        [41.8842],
        [43.9836],
        [42.6093],
        [43.6616],
        [43.3086],
        [41.5770],
        [44.2499],
        [42.8244],
        [42.5612],
        [42.7274],
        [40.

tensor([[42.9846],
        [42.8007],
        [41.2673],
        [42.7748],
        [42.8173],
        [42.7629],
        [43.5167],
        [41.6952],
        [43.7656],
        [44.6941],
        [43.7345],
        [42.9225],
        [43.9021],
        [43.8625],
        [43.3425],
        [42.8463],
        [42.7322],
        [42.1150],
        [40.2761],
        [42.7295],
        [41.9654],
        [42.8592],
        [42.9514],
        [43.6725],
        [42.4910],
        [40.9425],
        [42.0532],
        [41.6970],
        [42.6164],
        [41.4887],
        [39.9808],
        [42.4426],
        [41.5642],
        [42.3849],
        [40.7438],
        [40.7710],
        [44.5602],
        [44.6845],
        [41.2040],
        [41.9114],
        [40.6004],
        [43.4600],
        [42.7697],
        [43.6357],
        [42.6632],
        [44.4885],
        [43.7362],
        [40.0434],
        [42.8925],
        [44.3201],
        [43.4800],
        [44.3321],
        [44.

In [10]:
#predict
with torch.no_grad():
    hazard_rate_pred = Deep_CPH_model(X_val)

#predict

with torch.no_grad():
    predicted_survival = survival_model(X_val) / hazard_rate_pred.exp()
    general_prediction= predicted_survival.cpu().detach().numpy()
    print(general_prediction.max(), general_prediction.min(), general_prediction.shape)
    general_prediction[general_prediction >= 5000] = 5000
    
    print(abs(predicted_survival.reshape(-1)-Y_val).mean())
    
    c = concordance_correlation_coefficient(Y_val.cpu().detach().numpy(), general_prediction.reshape(-1))
    print("c: ", c)

42.347237 42.347187 (49, 1)
tensor(34.5568, device='cuda:0')
c:  -2.2224604502112057e-08


In [11]:
34.6412*7

242.48839999999998